In [1]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# This software may be used and distributed according to the terms of the Llama 2 Community License Agreement.

import torch
# Installation guide for cu11.8 and cu12.1 https://docs.vllm.ai/en/latest/getting_started/installation.html
from vllm import LLM
from vllm import LLM, SamplingParams
from llmformat.llminterface import build_vllm_logits_processor

torch.cuda.manual_seed(42)
torch.manual_seed(42)

def load_model(model_dir, tp_size=1):
    llm = LLM(model=model_dir, tensor_parallel_size=tp_size)
    return llm



In [2]:
def main(
    model,
    max_new_tokens=100,
    user_prompt=None,
    top_p=0.9,
    temperature=0.8
):
    while True:
        if user_prompt is None:
            user_prompt = input("Enter your prompt: ")
            
        print(f"User prompt:\n{user_prompt}")

        print(f"sampling params: top_p {top_p} and temperature {temperature} for this inference request")
        sampling_param = SamplingParams(top_p=top_p, 
                                        temperature=temperature, 
                                        max_tokens=max_new_tokens,
                                        logits_processors=[
                                            build_vllm_logits_processor(model, "/root/llmformat/llmformat/json.bnf")
                                        ])

        outputs = model.generate(user_prompt, sampling_params=sampling_param)
   
        print(f"model output:\n {user_prompt} {outputs[0].outputs[0].text}")
        user_prompt = input("Enter next prompt (press Enter to exit): ")
        if not user_prompt:
            break

def run_script(
    model_dir,
    tp_size=1,
    max_new_tokens=100,
    user_prompt=None,
    top_p=0.9,
    temperature=0.8
):
    model = load_model(model_dir, tp_size)
    main(model, max_new_tokens, user_prompt, top_p, temperature)


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftConfig, PeftModel
local_dir = "/root/model/llama-2-7b/"
run_script("meta-llama/Llama-2-7b-hf")

INFO 12-23 20:53:21 llm_engine.py:73] Initializing an LLM engine with config: model='meta-llama/Llama-2-7b-hf', tokenizer='meta-llama/Llama-2-7b-hf', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=False, seed=0)
